In [ ]:
import tensorflow as tf
import numpy as np
import PIL
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import imageio
from PIL import Image,ImageOps
from scipy import ndimage
from scipy.io import loadmat

%matplotlib inline

In [ ]:
import glob
import imageio
import time
from IPython import display
from skimage.io import imread
import os
from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import load_img ,img_to_array
import pickle

In [ ]:
list_file = os.listdir('dataset')
num_images=len(list_file)
train_images = np.array([resize(imread(os.path.join('dataset',file_name)), (64, 64,3)) for file_name in list_file])

In [ ]:
#after_reading_make pickle for fast  processing
#saving as pickle(only to de done once)
with open('train_images_dataset.pickle', 'wb') as f:
    pickle.dump({'images':train_images}, f)

In [ ]:
#opening pickle 
#with open('train_images_dataset.pickle', 'rb') as f:
#    images = pickle.load(f)
#train_images=images['images']

## Converting RGB TO CIELUV Using custom implementation 

In [ ]:
## custom_implementation(slow)

class rgb_cieluv:
    def __init__(self, img):
        self.r = img[:,:,0]
        self.r=self.r.reshape(-1)
        self.g = img[:,:,1]
        self.g=self.g.reshape(-1)
        self.b = img[:,:,2]
        self.b=self.b.reshape(-1)
        
    
    def rgb_xyz(self):
        #//sR, sG and sB (Standard RGB) input range = 0 ÷ 255
        #//X, Y and Z output refer to a D65/2° standard illuminant.
        #
        #var_R = ( sR / 255 )
        #var_G = ( sG / 255 )
        #var_B = ( sB / 255 )
        #
        #if ( var_R > 0.04045 ) var_R = ( ( var_R + 0.055 ) / 1.055 ) ^ 2.4
        #else                   var_R = var_R / 12.92
        #if ( var_G > 0.04045 ) var_G = ( ( var_G + 0.055 ) / 1.055 ) ^ 2.4
        #else                   var_G = var_G / 12.92
        #if ( var_B > 0.04045 ) var_B = ( ( var_B + 0.055 ) / 1.055 ) ^ 2.4
        #else                   var_B = var_B / 12.92
        #
        #var_R = var_R * 100
        #var_G = var_G * 100
        #var_B = var_B * 100
        #
        #X = var_R * 0.4124 + var_G * 0.3576 + var_B * 0.1805
        #Y = var_R * 0.2126 + var_G * 0.7152 + var_B * 0.0722
        #Z = var_R * 0.0193 + var_G * 0.1192 + var_B * 0.9505
        
        var_r = self.r
        var_g = self.g
        var_b = self.b
        for i in range(len(var_r)):
            if var_r[i] > 0.04045:
                var_r[i] =  ( ( var_r[i] + 0.055 ) / 1.055 ) ** 2.4
            else:
                var_r[i]= var_r[i]/12.92
            if var_g[i] > 0.04045:
                var_g[i] =  ( ( var_g[i] + 0.055 ) / 1.055 ) ** 2.4
            else:
                var_g[i] = var_g[i] /12.92
            if var_b[i] > 0.04045:
                var_b[i] =  ( ( var_b[i] + 0.055 ) / 1.055 ) ** 2.4
            else:
                var_b[i]= var_b[i]/12.92
              
        var_r*=100
        var_g*=100
        var_b*=100
        
        X = var_r * 0.4124 + var_g * 0.3576 + var_b * 0.1805
        Y = var_r * 0.2126 + var_g * 0.7152 + var_b * 0.0722
        Z = var_r * 0.0193 + var_g * 0.1192 + var_b * 0.9505
        
        return X,Y,Z
        

    def xyz_cieluv(self):
        X,Y,Z = self.rgb_xyz()
        var_u = ( 4 * X ) / ( X + ( 15 * Y ) + ( 3 * Z ) )
        var_v = ( 9 * Y ) / ( X + ( 15 * Y ) + ( 3 * Z ) )
        ref_y = Y / 100
        ReferenceX,ReferenceY,ReferenceZ=100,100,100
        ref_U = ( 4 * ReferenceX ) / ( ReferenceX + ( 15 * ReferenceY ) + ( 3 * ReferenceZ ) )
        ref_V = ( 9 * ReferenceY ) / ( ReferenceX + ( 15 * ReferenceY ) + ( 3 * ReferenceZ ) )
        CIEL=np.zeros(len(Y))
        for i in range(len(ref_y)):
            if ref_y[i]>0.008856:
                CIEL[i] = ( 116 * ref_y[i]**(1/3) ) - 16
            else:
                CIEL[i] = ref_y[i] * 903.3
        CIEu = 13 * CIEL * ( var_u - 0.2009 )
        CIEv = 13 * CIEL * ( var_v - 0.4610 )
        return CIEL,CIEu,CIEv
        
    def run(self):
        L,U,V=self.xyz_cieluv()
        L[np.isnan(L)] = 0
        U[np.isnan(U)] = 0
        V[np.isnan(V)] = 0
        return L,U,V

In [ ]:
## to run for custom implementation

train_L=[]
test_U=[]
test_V=[]
for each in train_images:
    L,U,V=rgb_cieluv(each).run()
    train_L.append(L)
    test_U.append(U)
    test_V.append(V)

In [ ]:
train_L=np.array(train_L).reshape(10000,64,64,1)
test_U=np.array(test_U).reshape(10000,64,64)
test_V=np.array(test_V).reshape(10000,64,64)

In [ ]:
with open('train_L.pickle', 'wb') as f:
    pickle.dump({'L':train_L}, f)
with open('test_U.pickle', 'wb') as f:
    pickle.dump({'U':np.array(test_U)}, f)
with open('test_V.pickle', 'wb') as f:
    pickle.dump({'V':test_V}, f)

In [ ]:
test={}
with open('test_V.pickle', 'rb') as f:
    test['V'] = pickle.load(f)
with open('test_U.pickle', 'rb') as f:
    test['U'] = pickle.load(f)
test['V']=np.array(test['V']['V'])
test['U']=np.array(test['U']['U'])
#test['UV']=np.transpose(np.array([test_U,test_V]),(1,2,3,0))
test['UV']=np.transpose(np.array([test['U'],test['V']]),(1,2,3,0))
#test['UV']=np.array(test['UV']).reshape(10000,224,224,2)

with open('test_UV.pickle', 'wb') as f:
    pickle.dump({'UV':test['UV']}, f)

## Converting rgb to cieluv(opencv based,fast)

In [ ]:
##open cv based implementation

import cv2
class rgb_luv_cv2:
    def __init__(self, images):
        self.images=images
        self.num_images=len(images)
    
    def rgb_l(self):
        train_L=[]
        i=0
        for each in self.images:
            each=each.astype('float32')
            luv = cv2.cvtColor(each,cv2.COLOR_RGB2Luv)
            train_L.append(luv[:,:,0])
            i+=1
            if i%500==0:
                print('Processing L channel Values for image ' +str(i))
        return np.array(train_L).reshape(self.num_images,64,64,1)
    
    def rgb_u(self):
        test_U=[]
        i=0
        for each in self.images:
            each=each.astype('float32')
            luv = cv2.cvtColor(each,cv2.COLOR_RGB2Luv)
            test_U.append(luv[:,:,1])
            i+=1
            if i%500==0:
                print('Processing U channel Values for image ' +str(i))
        return np.array(test_U).reshape(self.num_images,64,64)
    
    def rgb_v(self):
        test_V=[]
        i=0
        for each in self.images:
            each=each.astype('float32')
            luv = cv2.cvtColor(each,cv2.COLOR_RGB2Luv)
            test_V.append(luv[:,:,2])
            i+=1
            if i%500==0:
                print('Processing V channel Values for image ' +str(i))
        return np.array(test_V).reshape(self.num_images,64,64)
    
    def run(self):
        L=self.rgb_l()
        U=self.rgb_u()
        V=self.rgb_v()
        
        return L,U,V

In [ ]:
train_L,test_U,test_V=rgb_luv_cv2(train_images).run()

In [ ]:
with open('train_L.pickle', 'wb') as f:
    pickle.dump({'L':train_L}, f)
with open('test_U.pickle', 'wb') as f:
    pickle.dump({'U':np.array(test_U)}, f)
with open('test_V.pickle', 'wb') as f:
    pickle.dump({'V':test_V}, f)

In [ ]:
test={}
test['UV']=np.transpose(np.array([test_U,test_V]),(1,2,3,0))

In [ ]:
with open('test_UV.pickle', 'wb') as f:
    pickle.dump({'UV':test['UV']}, f)